In [232]:
from IPython.display import Latex

import numpy as np
from numpy.linalg import matrix_power, solve
from random import choices

from itertools import product
from inspect import isgeneratorfunction

import networkx as nx
from networkx.drawing.nx_agraph import to_agraph

### Методы

In [2]:
# Декоратор для округления возвращаемых значений других функций
def round_output(fn):
    rn = 5
    if isgeneratorfunction(fn): 
        def rounder(*args, **kwargs):
            for output in fn(*args, **kwargs):
                if type(output) != tuple:
                    yield np.round(output, rn)
                else:
                    res = []
                    for element in output:
                        res.append(np.round(element, rn))
                    yield tuple(res)
        return rounder
    else:
        def rounder(*args, **kwargs):
            output = fn(*args, **kwargs)
            return np.round(output, rn)  
        return rounder

In [8]:
def M2W(P):
    graph = {(str(i+1), str(j+1)): round(P[i][j], 5)
             for i, j in product(range(P.shape[0]), repeat=2) if P[i][j] > 0.}
    return graph

def plot_graph(P, path='graph'):
    graph = M2W(P)
    G=nx.MultiDiGraph()

    for edge in graph:
        G.add_edge(edge[0], edge[1])

    G.graph['edge'] = {'arrowsize': '1', 'splines': 'curved'}
    G.graph['graph'] = {'scale': '3'}

    A = to_agraph(G)
    A.layout('neato')#neato, dot, twopi, circo, fdp, nop, wc, acyclic, gvpr, gvcolor, ccomps, sccmap, tred, sfdp, unflatten

    for pair in graph:
        edge = A.get_edge(pair[0], pair[1])
        edge.attr['label'] = str(graph[pair]) + "  "

    A.draw(f'{path}.png')

In [6]:
def save_table_csv(table, path, columns, index, convert=True):
    df = pd.DataFrame(np.array(table).T if convert else table,
                      index=index,
                      columns=columns)
    df.to_csv(path+'.csv', sep=';', encoding='utf-8')

In [86]:
def matrix2latex(matr, toint=False):
    start = r'$$\begin{pmatrix} '
    end = r' \end{pmatrix}$$'
    if not toint:
        body = r' \\ '.join([r' & '.join([str(x) for x in matr[i]]) for i in range(matr.shape[0])]) 
    else:
        body = r' \\ '.join([r' & '.join([str(int(x)) for x in matr[i]]) for i in range(matr.shape[0])]) 
    return start + body + end

In [171]:
def ode_system2latex(P, toint=False):
    matr = P.T
    
    start = r'\[\left\{\begin{array}{ r @{{}={}} r  >{{}}c<{{}} r  >{{}}c<{{}}  r } '
    end = r' \end{array}\right.\]'
    
    frac = lambda i: r'\frac{dp_' + r'{0}'.format(i+1) + r'}{q}' + r'&=&'
    check_first = lambda arr, j: j == [i for i, x in enumerate(arr) if x != 0][0]
    def element(k, j, is_first):
        if not is_first:
            st = r'&+&' if k > 0 else r'&-&' 
        else:
            st = r''
            
        if not toint:
            if abs(k) != 1:
                bd = r'{0}'.format(abs(k)) if not is_first else r'{0}'.format(k) 
            else:
                bd = r''
                if is_first:
                    bd = r'' if k > 0 else r'-'
        else:
            if abs(k) != 1:
                bd = r'{0}'.format(abs(int(k))) if not is_first else r'{0}'.format(int(k)) 
            else:
                bd = r''
                if is_first:
                    bd = r'' if k > 0 else r'-'
    
            nd = r'p_{0}'.format(j+1)
        return st + bd + nd
    
    body = [frac(i) + r' '.join([element(matr[i][j], j, check_first(matr[i], j)) for j in range(matr.shape[1]) if matr[i][j] != 0]) + r'\\' 
            for i in range(matr.shape[0])]
    return start + r' '.join(body) + end

In [169]:
def stationary_system2latex(P, toint=False):
    matr = P.T
    
    start = r'\[\left\{\begin{array}{ r @{{}={}} r  >{{}}c<{{}} r  >{{}}c<{{}}  r } '
    end = r' \end{array}\right.\]'
    
    check_first = lambda arr, j: j == [i for i, x in enumerate(arr) if x != 0][0]
    
    def element(k, j, is_first):
        if not is_first:
            st = r'&+&' if k > 0 else r'&-&' 
        else:
            st = r''
            
        if not toint:
            if abs(k) != 1:
                bd = r'{0}'.format(abs(k)) if not is_first else r'{0}'.format(k) 
            else:
                bd = r''
                if is_first:
                    bd = r'' if k > 0 else r'-'
        else:
            if abs(k) != 1:
                bd = r'{0}'.format(abs(int(k))) if not is_first else r'{0}'.format(int(k)) 
            else:
                bd = r''
                if is_first:
                    bd = r'' if k > 0 else r'-'
    
            nd = r'p_{0}'.format(j+1)
        return st + bd + nd
    
    body = [r' '.join([element(matr[i][j], j, check_first(matr[i], j)) for j in range(matr.shape[1]) if matr[i][j] != 0]) + r' &=& 0' + r'\\' 
            for i in range(matr.shape[0])]
    body += [r' '.join([element(1.0, j, check_first(np.ones(matr.shape[1]), j)) for j in range(matr.shape[1])]) + r' &=& 1' + r'\\']
    return start + r' '.join(body) + end    

In [227]:
@round_output
def get_stationary_d(P):
    A = np.concatenate((P.T[:-1], np.ones((1, P.shape[0]))), axis=0)
    B = np.zeros(P.shape[0])
    B[-1] = 1.
    x = solve(A, B)
    return x

In [ ]:
def generate_next_state(current_state_id, P):
    return choices(population=[0, 1, 2], weights=P[current_state_id])[0]

@round_output
def generate_n_states(start_state_id, P, stationary, dm=1e-3):
    current_state_id = start_state_id
    count_start_state = 0
    N = dm * 2
    n = 0
    while N >= dm:
        current_state_id = generate_next_state(current_state_id, P)
        n += 1
        count_start_state += 1 if current_state_id == start_state_id else 0
        v = count_start_state / n
        N = abs(v - stationary[start_state_id])
        yield count_start_state, v, N
        
@round_output
def generate_n_states(P, stationary, dm=1e-4):
    current_state_id = 0
    K = 0
    vK = [0]
    RK = []

### 0) Подготовка

In [10]:
P = np.genfromtxt('Data/input.txt', comments="%")

In [87]:
display(Latex(matrix2latex(P, True)))

<IPython.core.display.Latex object>

### 1) Построить граф

In [16]:
plot_graph(P, 'Data/test')

### 2) СДУ Колмогорова

In [172]:
display(Latex(ode_system2latex(P, True)))

<IPython.core.display.Latex object>

### СУ для стационарных вероятностей

In [170]:
display(Latex(stationary_system2latex(P, True)))

<IPython.core.display.Latex object>

### Стационарное распределение

In [228]:
stationary = get_stationary_d(P)

In [231]:
display(Latex(matrix2latex(stationary.reshape(1, -1))))

<IPython.core.display.Latex object>